## Step 1 : Web Scraping 

In [1]:
#Import dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
import pandas as pd
from random import randint
from time import sleep 

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\eenab\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [3]:
#Create empty dictionary to hold all variables 
mars_data = {}

# title : string
# paragraph : string
# image :string
# pd.to_html -- run on the table they provide : string
# hemispheres: list of dictionaries


## NASA Mars News

In [4]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
sleep(randint(1,10))

In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')

In [6]:
#Find latest news title 
news_title = soup.find_all('div', class_='content_title')
news_title[1].get_text()
title = news_title[1].get_text()
mars_data["title"] = title

In [7]:
#Find latest news description
news_p = soup.find_all('div', class_='article_teaser_body')
news_p[1].get_text()
mars_data["news_p"] = news_p
news_p

[<div class="article_teaser_body">The agency’s newest rover captured first-of-its kind footage of its Feb. 18 touchdown and has recorded audio of Martian wind.
 
 </div>,
 <div class="article_teaser_body">First-of-its kind footage from the agency’s newest rover will be presented during a briefing this morning.</div>,
 <div class="article_teaser_body">The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. </div>,
 <div class="article_teaser_body">The six-wheeled robot’s latest data since touching down yesterday include a hi-res image captured as the rover’s jetpack lowered it to the ground.</div>,
 <div class="article_teaser_body">The agency’s latest and most complex mission to the Red Planet has touched down at Jezero Crater. Now it’s time to begin testing the health of the rover.  </div>,
 <div class="article_teaser_body">When the agency’s newest rover mission searches for fossilized microscopic life on the Red Planet, how wil

In [8]:
browser.quit()

## Featured Image

In [ ]:
#Find latest news photo
photo = soup.find_all('div', class_='list_image')
photo[0].find("img")["src"]

In [ ]:
#Create photo url
# 'https://mars.nasa.gov/system/news_items/list_view_images/8852_Mars2020-Landing-Technique-320x240.jpg'
base_url = 'https://mars.nasa.gov'
featured_image_url = base_url + photo[0].find("img")["src"]
mars_data["featured_image_url"] = featured_image_url

In [ ]:
#####JPL IMAGE

In [9]:
# Visit JPL Mars Space Images
jpl_img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_img_url)
# Sort for latest Mars photos in gallery 
# browser.find_by_css('.text-theme-red')[1].click() # finds and clicks the 'SORT BY' menu
# browser.find_by_tag('option')[2].click() # finds and clicks the 'LATEST' option
# browser.find_by_css('.SearchResultCard').first.click() # finds and clicks first/featured image


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61426): Max retries exceeded with url: /session/3fc822be0d1b187a87976f18633900b4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022B721E47C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# # Sort for latest Mars photos in gallery 
# browser.find_by_css('.text-theme-red')[1].click() # finds and clicks the 'SORT BY' menu
# browser.find_by_tag('option')[2].click() # finds and clicks the 'LATEST' option
# browser.find_by_css('.SearchResultCard').first.click() # finds and clicks first/featured image

In [ ]:
# partial_href = soup.find('div', class_='SearchResultCard').find('a', class_='group')['href']
#     browser.click_link_by_partial_href(partial_href)

In [ ]:
browser.quit()

## Mars Facts

In [ ]:
# Visit Mars facts url through splinter module
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
# sleep(randint(1,10))

In [ ]:
# Scrape the table containing facts about the planet including Diameter, Mass, etc.and turn into HTML table string
facts_table = pd.read_html(facts_url)
mars_data["facts_table"] = facts_table

In [ ]:
mars_data

## Mars Hemispheres

In [ ]:
hemisphere_image_urls = []
    # {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    # {"title": "Cerberus Hemisphere", "img_url": "..."},
    # {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    # {"title": "Syrtis Major Hemisphere", "img_url": "..."},


In [ ]:
# Visit Mars Hemispheres url through splinter module
hemi_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
browser.visit(hemi_url)
# sleep(randint(1,10))

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')

In [ ]:
valles_photo= soup.find_all('div', class_='downloads')
valles_photo[0].find

In [ ]:
# hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
# browser.visit(hemis_url)
# # sleep(randint(1,10))

In [ ]:
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

In [ ]:
base_url = "https://astrogeology.usgs.gov"
hemis_img_urls = []

results = soup.find_all("div", class_='description')
len(results)

In [ ]:
results[0].find('a')['href']

In [ ]:
for result in results:
    
    title = result.find('h3').text
    img_page_url = base_url + result.find('a')['href']
    
    #browser.click img_page_url
    response = requests.get(img_page_url)
    img_soup = BeautifulSoup(response.text, 'html.parser')
    
    img_url = img_soup.find('ul').li.a['href']
    
    img_dict = {'title': title, 
                'img_url': img_url}
    
    hemis_img_urls.append(img_dict)

    #browser.click and browser.back

In [ ]:
hemis_img_urls

In [ ]:
browser.quit()

In [ ]:
mars_data